In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time
import os

In [2]:
sampleSize=16000
sample_rate=16000
quantization_channels=256
#dilations=[2**i for i in range(8)]*20
#"residualDim=32
dilations=[2**i for i in range(9)]*1
residualDim=128
skipDim=512
filterSize=3
pad = np.sum(dilations)
lossrecord=[]
pad=0

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
#device = 'cpu'
torch.set_default_tensor_type('torch.cuda.FloatTensor')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [5]:
def mu_law_encode(audio, quantization_channels=quantization_channels,forX=False):
    '''Quantizes waveform amplitudes.'''
    mu = (quantization_channels - 1)*1.0
    # Perform mu-law companding transformation (ITU-T, 1988).
    # Minimum operation is here to deal with rare large amplitudes caused
    # by resampling.
    safe_audio_abs = np.minimum(np.abs(audio), 1.0)
    magnitude = np.log1p(mu * safe_audio_abs) / np.log1p(mu)
    signal = np.sign(audio) * magnitude
    # Quantize signal to the specified number of levels.
    if(forX):return signal
    return ((signal + 1) / 2 * mu + 0.5).astype(int)
def mu_law_decode(output, quantization_channels=quantization_channels):
    '''Recovers waveform from quantized values.'''
    mu = quantization_channels - 1
    # Map values back to [-1, 1].
    signal = 2 * ((output*1.0) / mu) - 1
    # Perform inverse of mu-law transformation.
    magnitude = (1 / mu) * ((1 + mu)**np.abs(signal) - 1)
    return np.sign(signal) * magnitude
def onehot(a,mu=quantization_channels):
    b = np.zeros((a.shape[0], mu))
    b[np.arange(a.shape[0]), a] = 1
    return b

In [6]:
def readAudio(name):
    audio0, samplerate = sf.read(name, dtype='float32')
    return librosa.resample(audio0.T, samplerate, sample_rate).reshape(-1)
p=['./vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav',
   './vsCorpus/origin_mix.wav','./vsCorpus/origin_vocal.wav','./vsCorpus/pred_mix.wav']
xtrain,ytrain,xval,yval,xtest=readAudio(p[0]),readAudio(p[1]),readAudio(p[2]),readAudio(p[3]),readAudio(p[4])
assert((xtrain==xval).all())
assert((ytrain==yval).all())
assert((xtrain != ytrain).any())

In [7]:
ytrain,yval=mu_law_encode(ytrain),mu_law_encode(yval)
xtrain,xval,xtest=mu_law_encode(xtrain,forX=True),mu_law_encode(xval,forX=True),mu_law_encode(xtest,forX=True)
assert (xtrain.max()<=2 and ytrain.max() >= 5)

In [8]:
'''xmean,xstd = xtrain.mean(),xtrain.std()
xtrain=(xtrain-xmean)/xstd
xval=(xval-xmean)/xstd
xtest=(xtest-xmean)/xstd'''

'xmean,xstd = xtrain.mean(),xtrain.std()\nxtrain=(xtrain-xmean)/xstd\nxval=(xval-xmean)/xstd\nxtest=(xtest-xmean)/xstd'

In [9]:
#xtrain=np.pad(xtrain, (pad, pad), 'constant')
#xval=np.pad(xval, (pad, pad), 'constant')
#xtest=np.pad(xtest, (pad, pad), 'constant')
#yval=np.pad(yval, (pad, pad), 'constant')
#ytrain=np.pad(ytrain, (pad, pad), 'constant')

In [10]:
xtrain,xval,xtest=xtrain.reshape(1,1,-1),xval.reshape(1,1,-1),xtest.reshape(1,1,-1)
ytrain,yval=ytrain.reshape(1,-1),yval.reshape(1,-1)

In [11]:
xtrain,ytrain,xval,yval,xtest = torch.from_numpy(xtrain).type(torch.float32),\
                                torch.from_numpy(ytrain).type(torch.LongTensor),\
                                torch.from_numpy(xval).type(torch.float32),\
                                torch.from_numpy(yval).type(torch.LongTensor),\
                                torch.from_numpy(xtest).type(torch.float32)

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        sd,qd,rd = skipDim,quantization_channels,residualDim
        self.causal = nn.Conv1d(in_channels=1,out_channels=rd,kernel_size=3,padding=1)
        self.tanhconvs = nn.ModuleList()
        self.sigmoidconvs = nn.ModuleList()
        self.skipconvs = nn.ModuleList()
        self.denseconvs = nn.ModuleList()
        for i, d in enumerate(dilations):
            self.tanhconvs.append(nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=d,dilation=d))
            self.sigmoidconvs.append(nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=3,padding=d,dilation=d))
            self.skipconvs.append(nn.Conv1d(in_channels=rd,out_channels=sd,kernel_size=1))
            self.denseconvs.append(nn.Conv1d(in_channels=rd,out_channels=rd,kernel_size=1))
        self.post1 = nn.Conv1d(in_channels=sd,out_channels=sd,kernel_size=1)
        self.post2 = nn.Conv1d(in_channels=sd,out_channels=qd,kernel_size=1)
        self.tanh,self.sigmoid = nn.Tanh(),nn.Sigmoid()

    def forward(self, x):
        finallen = x.shape[-1]-2*pad
        x = self.causal(x)
        for i, dilation in enumerate(dilations):
            xinput = x.clone()#[:,:,dilation:-dilation]
            x1 = self.tanh(self.tanhconvs[i](x))
            x2 = self.sigmoid(self.sigmoidconvs[i](x))
            x = x1*x2
            #cutlen = (x.shape[-1] - finallen)//2
            if(i == 0):skip_connections= (self.skipconvs[i](x))#.narrow(2,int(cutlen),int(finallen))
            else :skip_connections += (self.skipconvs[i](x))#.narrow(2,int(cutlen),int(finallen))
            x = self.denseconvs[i](x)
            x += xinput
        x = self.post2(F.relu(self.post1(F.relu(skip_connections))))
        return x

model = Net()#.cuda()
criterion = nn.CrossEntropyLoss()#.cuda()
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-5)

In [ ]:
def val():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        idx = np.arange(pad,xtrain.shape[-1]-pad-sampleSize,1000)
        np.random.shuffle(idx)
        data = xtrain[:,:,idx[0]-pad:pad+idx[0]+sampleSize].to(device)
        target = ytrain[:,idx[0]:idx[0]+sampleSize].to(device)
        output = model(data)
        pred = output.max(1, keepdim=True)[1]
        correct = pred.eq(target.view_as(pred)).sum().item() / pred.shape[-1]
        val_loss = criterion(output, target).item()
    print(correct,'accurate')
    print('\nval set:loss{:.4f}:, ({:.3f} sec/step)\n'.format(val_loss,time.time()-startval_time))

def test():
    model.eval()
    startval_time = time.time()
    with torch.no_grad():
        listofpred = []
        for ind in range(pad, xtest.shape[-1] - pad, sampleSize):
            output = model(xtest[:, :, ind - pad:ind + sampleSize + pad].to(device))
            pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
            listofpred.append(pred)
        ans = mu_law_decode(np.concatenate(listofpred))
        sf.write('./vsCorpus/resultxte.wav', ans, sample_rate)

        listofpred=[]
        for ind in range(pad,xtrain.shape[-1]-pad,sampleSize):
            output = model(xtrain[:, :, ind-pad:ind+sampleSize+pad].to(device))
            pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
            listofpred.append(pred)
        ans = mu_law_decode(np.concatenate(listofpred))
        sf.write('./vsCorpus/resultxtr.wav', ans, sample_rate)
        print('stored done\n')
    
def train(epoch):
    model.train()
    idx = np.arange(pad,xtrain.shape[-1]-pad-sampleSize,16000)
    np.random.shuffle(idx)
    for i, ind in enumerate(idx):
        start_time = time.time()
        data, target = xtrain[:,:,ind-pad:ind+sampleSize+pad].to(device), ytrain[:,ind:ind+sampleSize].to(device)
        output = model(data)
        loss = criterion(output, target)
        lossrecord.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)] Loss:{:.6f}: , ({:.3f} sec/step)'.format(
                epoch, i, len(idx),100. * i / len(idx), loss.item(),time.time() - start_time))
        if i % 100 == 0:
            with open("./lossRecord/newmodellossfile.txt", "w") as f:
                for s in lossrecord:
                    f.write(str(s) +"\n")
            print('write finish')
            state={'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()}
            torch.save(state, 'newModifiedModel')
    test()
    val()

In [ ]:
for epoch in range(100000):
    train(epoch)
    test()

Train Epoch: 0 [0/235 (0%)] Loss:5.547755: , (0.486 sec/step)
write finish
Train Epoch: 0 [1/235 (0%)] Loss:5.421005: , (0.481 sec/step)
Train Epoch: 0 [2/235 (1%)] Loss:4.892072: , (0.481 sec/step)
Train Epoch: 0 [3/235 (1%)] Loss:7.993630: , (0.482 sec/step)
Train Epoch: 0 [4/235 (2%)] Loss:6.737203: , (0.481 sec/step)
Train Epoch: 0 [5/235 (2%)] Loss:5.817010: , (0.481 sec/step)
Train Epoch: 0 [6/235 (3%)] Loss:5.522730: , (0.481 sec/step)
Train Epoch: 0 [7/235 (3%)] Loss:5.525379: , (0.481 sec/step)
Train Epoch: 0 [8/235 (3%)] Loss:5.524930: , (0.481 sec/step)
Train Epoch: 0 [9/235 (4%)] Loss:5.505610: , (0.481 sec/step)
Train Epoch: 0 [10/235 (4%)] Loss:5.487419: , (0.480 sec/step)
Train Epoch: 0 [11/235 (5%)] Loss:5.456210: , (0.481 sec/step)
Train Epoch: 0 [12/235 (5%)] Loss:5.477621: , (0.481 sec/step)
Train Epoch: 0 [13/235 (6%)] Loss:5.360456: , (0.482 sec/step)
Train Epoch: 0 [14/235 (6%)] Loss:5.380502: , (0.481 sec/step)
Train Epoch: 0 [15/235 (6%)] Loss:5.263237: , (0.481

Train Epoch: 0 [128/235 (54%)] Loss:4.758199: , (0.484 sec/step)
Train Epoch: 0 [129/235 (55%)] Loss:4.636548: , (0.484 sec/step)
Train Epoch: 0 [130/235 (55%)] Loss:4.748213: , (0.484 sec/step)
Train Epoch: 0 [131/235 (56%)] Loss:4.537552: , (0.484 sec/step)
Train Epoch: 0 [132/235 (56%)] Loss:4.678299: , (0.484 sec/step)
Train Epoch: 0 [133/235 (57%)] Loss:4.725358: , (0.485 sec/step)
Train Epoch: 0 [134/235 (57%)] Loss:4.405289: , (0.484 sec/step)
Train Epoch: 0 [135/235 (57%)] Loss:5.140233: , (0.484 sec/step)
Train Epoch: 0 [136/235 (58%)] Loss:5.322268: , (0.484 sec/step)
Train Epoch: 0 [137/235 (58%)] Loss:4.159271: , (0.484 sec/step)
Train Epoch: 0 [138/235 (59%)] Loss:4.686876: , (0.484 sec/step)
Train Epoch: 0 [139/235 (59%)] Loss:4.660340: , (0.484 sec/step)
Train Epoch: 0 [140/235 (60%)] Loss:4.958701: , (0.484 sec/step)
Train Epoch: 0 [141/235 (60%)] Loss:4.665672: , (0.484 sec/step)
Train Epoch: 0 [142/235 (60%)] Loss:5.022137: , (0.484 sec/step)
Train Epoch: 0 [143/235 (

Train Epoch: 1 [19/235 (8%)] Loss:4.733106: , (0.483 sec/step)
Train Epoch: 1 [20/235 (9%)] Loss:4.559147: , (0.483 sec/step)
Train Epoch: 1 [21/235 (9%)] Loss:4.597474: , (0.483 sec/step)
Train Epoch: 1 [22/235 (9%)] Loss:4.506425: , (0.483 sec/step)
Train Epoch: 1 [23/235 (10%)] Loss:4.267164: , (0.483 sec/step)
Train Epoch: 1 [24/235 (10%)] Loss:4.703687: , (0.482 sec/step)
Train Epoch: 1 [25/235 (11%)] Loss:4.433853: , (0.483 sec/step)
Train Epoch: 1 [26/235 (11%)] Loss:4.852136: , (0.482 sec/step)
Train Epoch: 1 [27/235 (11%)] Loss:4.619339: , (0.483 sec/step)
Train Epoch: 1 [28/235 (12%)] Loss:4.850651: , (0.483 sec/step)
Train Epoch: 1 [29/235 (12%)] Loss:4.602393: , (0.483 sec/step)
Train Epoch: 1 [30/235 (13%)] Loss:4.431938: , (0.482 sec/step)
Train Epoch: 1 [31/235 (13%)] Loss:4.494323: , (0.483 sec/step)
Train Epoch: 1 [32/235 (14%)] Loss:4.824885: , (0.482 sec/step)
Train Epoch: 1 [33/235 (14%)] Loss:4.458988: , (0.484 sec/step)
Train Epoch: 1 [34/235 (14%)] Loss:4.611473:

Train Epoch: 1 [147/235 (63%)] Loss:4.569782: , (0.483 sec/step)
Train Epoch: 1 [148/235 (63%)] Loss:4.866638: , (0.483 sec/step)
Train Epoch: 1 [149/235 (63%)] Loss:4.313173: , (0.482 sec/step)
Train Epoch: 1 [150/235 (64%)] Loss:4.614216: , (0.483 sec/step)
Train Epoch: 1 [151/235 (64%)] Loss:4.396182: , (0.483 sec/step)
Train Epoch: 1 [152/235 (65%)] Loss:4.492830: , (0.483 sec/step)
Train Epoch: 1 [153/235 (65%)] Loss:5.021877: , (0.484 sec/step)
Train Epoch: 1 [154/235 (66%)] Loss:4.592732: , (0.482 sec/step)
Train Epoch: 1 [155/235 (66%)] Loss:4.463941: , (0.483 sec/step)
Train Epoch: 1 [156/235 (66%)] Loss:4.801547: , (0.483 sec/step)
Train Epoch: 1 [157/235 (67%)] Loss:4.451128: , (0.483 sec/step)
Train Epoch: 1 [158/235 (67%)] Loss:4.516891: , (0.482 sec/step)
Train Epoch: 1 [159/235 (68%)] Loss:4.453228: , (0.482 sec/step)
Train Epoch: 1 [160/235 (68%)] Loss:4.407773: , (0.483 sec/step)
Train Epoch: 1 [161/235 (69%)] Loss:5.103195: , (0.483 sec/step)
Train Epoch: 1 [162/235 (

Train Epoch: 2 [38/235 (16%)] Loss:4.677846: , (0.484 sec/step)
Train Epoch: 2 [39/235 (17%)] Loss:4.189075: , (0.485 sec/step)
Train Epoch: 2 [40/235 (17%)] Loss:5.264687: , (0.484 sec/step)
Train Epoch: 2 [41/235 (17%)] Loss:4.635754: , (0.484 sec/step)
Train Epoch: 2 [42/235 (18%)] Loss:4.600896: , (0.485 sec/step)
Train Epoch: 2 [43/235 (18%)] Loss:4.492909: , (0.485 sec/step)
Train Epoch: 2 [44/235 (19%)] Loss:4.821988: , (0.486 sec/step)
Train Epoch: 2 [45/235 (19%)] Loss:4.697877: , (0.485 sec/step)
Train Epoch: 2 [46/235 (20%)] Loss:4.412129: , (0.485 sec/step)
Train Epoch: 2 [47/235 (20%)] Loss:4.546309: , (0.485 sec/step)
Train Epoch: 2 [48/235 (20%)] Loss:4.792161: , (0.485 sec/step)
Train Epoch: 2 [49/235 (21%)] Loss:4.333576: , (0.485 sec/step)
Train Epoch: 2 [50/235 (21%)] Loss:4.268005: , (0.484 sec/step)
Train Epoch: 2 [51/235 (22%)] Loss:4.408892: , (0.484 sec/step)
Train Epoch: 2 [52/235 (22%)] Loss:4.351143: , (0.485 sec/step)
Train Epoch: 2 [53/235 (23%)] Loss:4.548

...

Train Epoch: 6 [111/235 (47%)] Loss:4.427845: , (0.485 sec/step)
Train Epoch: 6 [112/235 (48%)] Loss:4.361942: , (0.484 sec/step)
Train Epoch: 6 [113/235 (48%)] Loss:4.877975: , (0.485 sec/step)
Train Epoch: 6 [114/235 (49%)] Loss:4.368456: , (0.484 sec/step)
Train Epoch: 6 [115/235 (49%)] Loss:4.592480: , (0.485 sec/step)
Train Epoch: 6 [116/235 (49%)] Loss:4.298288: , (0.485 sec/step)
Train Epoch: 6 [117/235 (50%)] Loss:4.734350: , (0.485 sec/step)
Train Epoch: 6 [118/235 (50%)] Loss:4.814129: , (0.484 sec/step)
Train Epoch: 6 [119/235 (51%)] Loss:4.257486: , (0.485 sec/step)
Train Epoch: 6 [120/235 (51%)] Loss:4.758906: , (0.485 sec/step)
Train Epoch: 6 [121/235 (51%)] Loss:4.590443: , (0.484 sec/step)
Train Epoch: 6 [122/235 (52%)] Loss:4.115832: , (0.483 sec/step)
Train Epoch: 6 [123/235 (52%)] Loss:4.092295: , (0.484 sec/step)
Train Epoch: 6 [124/235 (53%)] Loss:4.192016: , (0.484 sec/step)
Train Epoch: 6 [125/235 (53%)] Loss:4.132867: , (0.486 sec/step)
Train Epoch: 6 [126/235 (

Train Epoch: 7 [1/235 (0%)] Loss:3.911591: , (0.484 sec/step)
Train Epoch: 7 [2/235 (1%)] Loss:4.180709: , (0.483 sec/step)
Train Epoch: 7 [3/235 (1%)] Loss:4.446379: , (0.484 sec/step)
Train Epoch: 7 [4/235 (2%)] Loss:4.415327: , (0.484 sec/step)
Train Epoch: 7 [5/235 (2%)] Loss:4.033854: , (0.484 sec/step)
Train Epoch: 7 [6/235 (3%)] Loss:4.288366: , (0.484 sec/step)
Train Epoch: 7 [7/235 (3%)] Loss:4.045988: , (0.484 sec/step)
Train Epoch: 7 [8/235 (3%)] Loss:4.287700: , (0.484 sec/step)
Train Epoch: 7 [9/235 (4%)] Loss:4.124026: , (0.484 sec/step)
Train Epoch: 7 [10/235 (4%)] Loss:4.215674: , (0.484 sec/step)
Train Epoch: 7 [11/235 (5%)] Loss:4.179538: , (0.485 sec/step)
Train Epoch: 7 [12/235 (5%)] Loss:3.961470: , (0.484 sec/step)
Train Epoch: 7 [13/235 (6%)] Loss:4.163476: , (0.484 sec/step)
Train Epoch: 7 [14/235 (6%)] Loss:4.325493: , (0.484 sec/step)
Train Epoch: 7 [15/235 (6%)] Loss:4.373833: , (0.484 sec/step)
Train Epoch: 7 [16/235 (7%)] Loss:3.906120: , (0.485 sec/step)
T

Train Epoch: 7 [129/235 (55%)] Loss:3.791935: , (0.486 sec/step)
Train Epoch: 7 [130/235 (55%)] Loss:4.588972: , (0.485 sec/step)
Train Epoch: 7 [131/235 (56%)] Loss:4.004332: , (0.485 sec/step)
Train Epoch: 7 [132/235 (56%)] Loss:3.835242: , (0.485 sec/step)
Train Epoch: 7 [133/235 (57%)] Loss:4.023383: , (0.485 sec/step)
Train Epoch: 7 [134/235 (57%)] Loss:4.079592: , (0.485 sec/step)
Train Epoch: 7 [135/235 (57%)] Loss:4.001272: , (0.485 sec/step)
Train Epoch: 7 [136/235 (58%)] Loss:4.164142: , (0.485 sec/step)
Train Epoch: 7 [137/235 (58%)] Loss:3.967960: , (0.486 sec/step)
Train Epoch: 7 [138/235 (59%)] Loss:4.088146: , (0.485 sec/step)
Train Epoch: 7 [139/235 (59%)] Loss:4.058914: , (0.486 sec/step)
Train Epoch: 7 [140/235 (60%)] Loss:4.726306: , (0.485 sec/step)
Train Epoch: 7 [141/235 (60%)] Loss:4.450330: , (0.485 sec/step)
Train Epoch: 7 [142/235 (60%)] Loss:3.938257: , (0.485 sec/step)
Train Epoch: 7 [143/235 (61%)] Loss:4.329419: , (0.485 sec/step)
Train Epoch: 7 [144/235 (

Train Epoch: 8 [20/235 (9%)] Loss:4.445338: , (0.485 sec/step)
Train Epoch: 8 [21/235 (9%)] Loss:4.368414: , (0.486 sec/step)
Train Epoch: 8 [22/235 (9%)] Loss:4.200591: , (0.485 sec/step)
Train Epoch: 8 [23/235 (10%)] Loss:4.579358: , (0.485 sec/step)
Train Epoch: 8 [24/235 (10%)] Loss:3.895213: , (0.486 sec/step)
Train Epoch: 8 [25/235 (11%)] Loss:3.973351: , (0.486 sec/step)
Train Epoch: 8 [26/235 (11%)] Loss:4.304497: , (0.485 sec/step)
Train Epoch: 8 [27/235 (11%)] Loss:4.237315: , (0.486 sec/step)
Train Epoch: 8 [28/235 (12%)] Loss:4.268793: , (0.486 sec/step)
Train Epoch: 8 [29/235 (12%)] Loss:4.889627: , (0.486 sec/step)
Train Epoch: 8 [30/235 (13%)] Loss:3.957901: , (0.486 sec/step)
Train Epoch: 8 [31/235 (13%)] Loss:4.902222: , (0.486 sec/step)
Train Epoch: 8 [32/235 (14%)] Loss:4.373780: , (0.486 sec/step)
Train Epoch: 8 [33/235 (14%)] Loss:4.004298: , (0.486 sec/step)
Train Epoch: 8 [34/235 (14%)] Loss:4.258637: , (0.486 sec/step)
Train Epoch: 8 [35/235 (15%)] Loss:4.366886

Train Epoch: 8 [148/235 (63%)] Loss:4.641431: , (0.484 sec/step)
Train Epoch: 8 [149/235 (63%)] Loss:4.334437: , (0.485 sec/step)
Train Epoch: 8 [150/235 (64%)] Loss:3.963598: , (0.486 sec/step)
Train Epoch: 8 [151/235 (64%)] Loss:4.408089: , (0.485 sec/step)
Train Epoch: 8 [152/235 (65%)] Loss:4.087134: , (0.484 sec/step)
Train Epoch: 8 [153/235 (65%)] Loss:4.099693: , (0.485 sec/step)
Train Epoch: 8 [154/235 (66%)] Loss:4.273084: , (0.485 sec/step)
Train Epoch: 8 [155/235 (66%)] Loss:5.042213: , (0.485 sec/step)
Train Epoch: 8 [156/235 (66%)] Loss:3.962729: , (0.485 sec/step)
Train Epoch: 8 [157/235 (67%)] Loss:4.180250: , (0.485 sec/step)
Train Epoch: 8 [158/235 (67%)] Loss:4.086083: , (0.485 sec/step)
Train Epoch: 8 [159/235 (68%)] Loss:4.247157: , (0.485 sec/step)
Train Epoch: 8 [160/235 (68%)] Loss:4.474509: , (0.485 sec/step)
Train Epoch: 8 [161/235 (69%)] Loss:4.148944: , (0.485 sec/step)
Train Epoch: 8 [162/235 (69%)] Loss:4.075963: , (0.484 sec/step)
Train Epoch: 8 [163/235 (

In [ ]:
#torch.save(model, 'loss2.5~3.5step10h_repeat5*2**9resu32sample50000')